In [108]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime, os

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')
# df = pd.read_csv("news.csv") 

# 训练部分

In [115]:
#import random
#
#df = df[['title','content']]
#df["content"] = df["content"].apply(lambda x: x.strip('{').strip('}'))
#df['text'] = df['title'] + " " + df['content']
#df['y'] = np.random.randint(0, 2, size=len(df))
#df = df.sample(frac=1).reset_index(drop=True)
#df_raw = df[['text', 'y']]
#
#df_raw = df_raw[0:400]
#df_raw.to_csv("haha.csv")
#df_raw.head(20)

df = pd.read_csv("./train_data/train.csv")
df_raw = df[['text','y']]
df_raw = df_raw[df_raw['y'] != 0]
df_raw.loc[df_raw['y'] == -1, 'y'] = 0
df_raw.head()

,text,y
0,"人民创投携手游族网络、中科曙光揭牌成立 “新质生产力数字化创新联盟” 5月11日下午,由人民...",1
1,航天动力(600343.SH)及高管因虚假信息披露遭上交所处分 2024年4月2日，上海证券...,0
2,中国重工：2023年度净利润约-7.82亿元 中国重工（SH601989，收盘价：4.94元...,0
3,鲁 泰Ａ：公司产业链较长 各生产环节均存在期末在产品 每经AI快讯，有投资者在投资者互动平台...,0
4,方正电机：2024年第一季度净利润约-834万元 方正电机（SZ002196，收盘价：5.5...,0


In [116]:
import transformers
MODEL_PATH = r"/home/aiia/newDisk/q1w2e3r4/sentimics"

num_classes = 2 # only pos and neg
# a.通过词典导入分词器
tokenizer = transformers.BertTokenizer.from_pretrained(r"/home/aiia/newDisk/q1w2e3r4/sentimics/vocab.txt") 
# b. 导入配置文件
model_config = transformers.BertConfig.from_pretrained(MODEL_PATH)
# 修改配置
model_config.output_hidden_states = True
model_config.output_attentions = True
model_config.num_labels = num_classes
# 通过配置和路径导入模型
model = transformers.TFBertForSequenceClassification.from_pretrained(MODEL_PATH,config = model_config)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at /home/aiia/newDisk/q1w2e3r4/sentimics and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [117]:
max_length_test = 400

test_sentence_with_special_tokens = '[CLS]' + df['text'][0][:max_length_test] + '[SEP]'
tokenized = tokenizer.tokenize(test_sentence_with_special_tokens)
print('tokenized', tokenized)

input_ids = tokenizer.convert_tokens_to_ids(tokenized)

padding_length = max_length_test - len(input_ids)

input_ids = input_ids + ([0] * padding_length)

attention_mask = [1] * len(input_ids)

attention_mask = attention_mask + ([0] * padding_length)

token_type_ids = [0] * max_length_test
bert_input = {
    "token_ids": input_ids,
    "token_type_ids": token_type_ids,
    "attention_mask": attention_mask
} 

print(bert_input)

tokenized ['[CLS]', '人', '民', '创', '投', '携', '手', '游', '族', '网', '络', '、', '中', '科', '曙', '光', '揭', '牌', '成', '立', '[UNK]', '新', '质', '生', '产', '力', '数', '字', '化', '创', '新', '联', '盟', '[UNK]', '5', '月', '11', '日', '下', '午', ',', '由', '人', '民', '网', '人', '民', '创', '投', '、', '中', '科', '曙', '光', '、', '游', '族', '网', '络', '共', '同', '发', '起', '的', '新', '质', '生', '产', '力', '数', '字', '化', '创', '新', '联', '盟', '正', '式', '成', '立', '并', '举', '行', '揭', '牌', '仪', '式', '。', ';', '联', '盟', '相', '关', '负', '责', '人', '表', '示', ',', '随', '着', '人', '工', '智', '能', '等', '新', '一', '代', '信', '息', '技', '术', '的', '高', '速', '发', '展', ',', '由', '技', '术', '革', '命', '性', '突', '破', '、', '生', '产', '要', '素', '创', '新', '性', '配', '置', '、', '产', '业', '转', '型', '升', '级', '而', '催', '生', '的', '先', '进', '生', '产', '力', '质', '态', '加', '速', '形', '成', ',', '代', '表', '先', '进', '生', '产', '力', '演', '进', '方', '向', '的', '新', '质', '生', '产', '力', '应', '运', '而', '生', '。', '为', '建', '立', '一', '个', '促', '进', '产', '业', '界', '、', '学', '术', '

In [118]:
from sklearn.model_selection import train_test_split

def convert_example_to_feature(review):
    return tokenizer.encode_plus(review, 
                add_special_tokens = True, 
                max_length = max_length, 
                pad_to_max_length = True, 
                return_attention_mask = True, 
                truncation=True)


def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(ds, limit=-1):

    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    if (limit > 0):
        ds = ds.take(limit)

    for index, row in ds.iterrows():
        review = row["text"]
        label = row["y"]
        bert_input = convert_example_to_feature(review)

        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

def split_dataset(df):
    train_set, x = train_test_split(df, 
        stratify=df['y'],
        test_size=0.1, 
        random_state=42)
    val_set, test_set = train_test_split(x, 
        stratify=x['y'],
        test_size=0.5, 
        random_state=43)

    return train_set,val_set, test_set

max_length = 400
batch_size = 128

# split data
train_data,val_data, test_data = split_dataset(df_raw)

# train dataset
ds_train_encoded = encode_examples(train_data).batch(batch_size)
# val dataset
ds_val_encoded = encode_examples(val_data).batch(batch_size)
# test dataset
ds_test_encoded = encode_examples(test_data).batch(batch_size)

In [119]:
learning_rate = 2e-5
number_of_epochs = 8

# 三件套
# optimizer Adam recommended
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,epsilon=1e-08, clipnorm=1)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
model.summary()

Model: "tf_bert_for_sequence_classification_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  102267648 
                                                                 
 dropout_227 (Dropout)       multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 102269186 (390.13 MB)
Trainable params: 102269186 (390.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [120]:
bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_val_encoded)
# evaluate test_set
print("# evaluate test_set:",model.evaluate(ds_test_encoded))

Epoch 1/8
3/3 [==============================] - 91s 18s/step - loss: 0.7034 - accuracy: 0.5353 - val_loss: 0.5418 - val_accuracy: 0.7333
Epoch 2/8
3/3 [==============================] - 55s 15s/step - loss: 0.5953 - accuracy: 0.6989 - val_loss: 0.5199 - val_accuracy: 0.7333
Epoch 3/8
3/3 [==============================] - 56s 15s/step - loss: 0.5477 - accuracy: 0.7100 - val_loss: 0.5098 - val_accuracy: 0.8000
Epoch 4/8
3/3 [==============================] - 58s 15s/step - loss: 0.5094 - accuracy: 0.8067 - val_loss: 0.4576 - val_accuracy: 0.8667
Epoch 5/8
3/3 [==============================] - 58s 15s/step - loss: 0.4401 - accuracy: 0.8550 - val_loss: 0.3979 - val_accuracy: 0.8667
Epoch 6/8
3/3 [==============================] - 57s 15s/step - loss: 0.3809 - accuracy: 0.8625 - val_loss: 0.3215 - val_accuracy: 0.8667
Epoch 7/8
3/3 [==============================] - 58s 15s/step - loss: 0.3035 - accuracy: 0.9071 - val_loss: 0.2420 - val_accuracy: 0.9333
Epoch 8/8
1/1 [===================

In [121]:
tokenizer.encode_plus("我喜欢红色")

{'input_ids': [101, 2769, 1599, 3614, 5273, 5682, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [122]:
model.summary()

Model: "tf_bert_for_sequence_classification_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  102267648 
                                                                 
 dropout_227 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 102269186 (390.13 MB)
Trainable params: 102269186 (390.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# 预测部分，如果使用加载模型请从这里开始

In [176]:
# 加载训练好的模型
'''
model = tf.keras.models.load_model("my_model")

num_classes = 2
model_config = transformers.BertConfig.from_pretrained(MODEL_PATH)
model_config.output_hidden_states = True
model_config.output_attentions = True
model_config.num_labels = num_classes

model.config = model_config
model.summary()
'''

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "directionality": "bidi",
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_attentions": true,
  "output_hidden_states": true,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

Model: "tf_bert_for_sequence_classification_5"
_______________________________________________________________

In [175]:
from transformers import pipeline

# AutoTokenizer用于tokenize，可以通俗理解为分词
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

AttributeError: 'TFBertForSequenceClassification' object has no attribute 'can_generate'

In [ ]:
def sentiment_analyze(text):    
    scores = []
    sub_len = len(text)//400+1
    for i in range(sub_len):
        sub_text = text[i*400:(i+1)*400]
        res = classifier(sub_text)[0]
        s = res['score']
        if res['label'] == 'LABEL_0':  # neg
            s = -s
        scores.append(s)
    
    return np.mean(scores)

In [ ]:
#text = ''' 人民创投携手游族网络、中科曙光揭牌成立 “新质生产力数字化创新联盟” '''
#text = '''不好看，电影院里睡了三个小时'''
#text = '''航天动力(600343.SH)及高管因虚假信息披露遭上交所处分'''
text = '''2024年以来，启迪环境已连发4次关于其全资子公司、子公司收到《行政处罚决定书》的公告'''

sentiment_analyze(text)

In [ ]:
# now we test it
df = pd.read_csv("news.csv") 

In [ ]:
df["content"] = df["content"].apply(lambda x: x.strip('{').strip('}'))
df['text'] = df['title'] + df['content']
df.head()

In [ ]:
FIXED_LEN = 100
def is_chinese(uchar):
    # 判断当前字符是否为中文字符
    return uchar >= u'\u4e00' and uchar <= u'\u9fa5'
    
def strlen(s):
    ret = 0
    for c in s:
        if is_chinese(c):
            ret += 2
        else:
            ret += 1
    return ret
    
def ljust(s):
    l = strlen(s)
    padding = max(0, FIXED_LEN - l)
    return s + ' ' * padding
    
for index,row in df.iterrows():
    text = row['text']
    print("%s\t%f"%(ljust(row['title']), sentiment_analyze(text)))

In [177]:
model.save_pretrained('my_model')

AttributeError: 'TFBertForSequenceClassification' object has no attribute 'save_pretrained'

Model: "tf_bert_for_sequence_classification_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  102267648 
                                                                 
 dropout_227 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 102269186 (390.13 MB)
Trainable params: 102269186 (390.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
